# Assignment 6

First we have to create some SQL tables.

In [2]:
CREATE DATABASE MouseHypothalamus;

: Msg 1801, Level 16, State 3, Line 1
Database 'MouseHypothalamus' already exists. Choose a different database name.

Total execution time: 00:00:00.002

In [1]:
USE MouseHypothalamus;
DROP TABLE IF EXISTS #TempMolecules;
CREATE TABLE #TempMolecules(
    Gene_name NVARCHAR(10) NOT NULL,
    Cell_name NVARCHAR(40),
    Animal_ID int,
    Bregma float,
    Animal_sex nvarchar(6) NOT NULL,
    Behavior nvarchar(20),
    Centroid_X float NOT NULL,
    Centroid_Y float NOT NULL,
    Centroid_Z float NOT NULL,
    Total_brightness float,
    Area int,
    Error_bit tinyint,
    Error_direction tinyint
);

BULK INSERT #TempMolecules FROM '/var/data/merfish_barcodes.csv'
WITH ( 
    FIRSTROW = 2, -- skip the column headers
    --ROWS_PER_BATCH = 467052741, -- however many total rows the data has
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '0x0a',
    KEEPNULLS
);

SELECT IDENTITY(int,1,1) as id,* INTO Molecules FROM #TempMolecules;

(467052740 rows affected)

Total execution time: 00:58:33.595

TODO: Add a column for the identity (auto-increment).

In [3]:
Use MouseHypothalamus;
SELECT IDENTITY(int,1,1) as id,* INTO Molecules FROM #TempMolecules;

: Msg 208, Level 16, State 0, Line 2
Invalid object name '#TempMolecules'.

Total execution time: 00:00:00.004

# Geometries

Let's start by creating the geometry objects for the molecules.

In [9]:
Use MouseHypothalamus;

DROP TABLE IF EXISTS MoleculesWithPoints;
SELECT 
        id, 
        Gene_name, 
        Cell_name, 
        Animal_ID, 
        Bregma, 
        Centroid_Z,
        geometry::STGeomFromText('POINT(' + CONVERT(VARCHAR(20), Centroid_X) + ' ' + CONVERT(VARCHAR(20), Centroid_Y) + ')', 0) AS point
    INTO MoleculesWithPoints 
    FROM Molecules;


(467052740 rows affected)

Total execution time: 04:10:57.025

In [ ]:
ALTER TABLE MoleculesWithPointsHead
ADD CONSTRAINT moleculesWithPointsHead_id_PK PRIMARY KEY(id);

In [ ]:
ALTER TABLE MoleculesWithPoints
ADD CONSTRAINT moleculesWithPoints_id_PK PRIMARY KEY(id);

In [19]:
Use MouseHypothalamus;
SELECT Max(Centroid_X) as maxx, Max(Centroid_Y) as maxy,  Min(Centroid_X) as minx, Min(Centroid_Y) as miny FROM MoleculesHead;


(1 row affected)

Total execution time: 00:00:00.012

maxx,maxy,minx,miny
-2819.077,3387.196,-3017.902,3187.312


In [20]:
Use MouseHypothalamus;
SELECT Max(Centroid_X) as maxx, Max(Centroid_Y) as maxy,  Min(Centroid_X) as minx, Min(Centroid_Y) as miny FROM Molecules;

(1 row affected)

Total execution time: 00:00:03.816

maxx,maxy,minx,miny
4203.399,4968.7,-4289.699,-5009.986


In [21]:
Use MouseHypothalamus;
CREATE SPATIAL INDEX MoleculesPointInd ON
   [MouseHypothalamus].[dbo].[MoleculesWithPoints](point)
   WITH (GRIDS = (HIGH, HIGH, HIGH, HIGH), 
        BOUNDING_BOX = (XMIN = -4300,YMIN = -5100, XMAX = 4300, YMAX = 5000)); -- approx based on the actual min/max values in the data

: Msg 12008, Level 16, State 1, Line 2
Table 'MouseHypothalamus.dbo.MoleculesWithPoints' does not have a clustered primary key as required by the spatial index. Make sure that the primary key column exists on the table before creating a spatial index.

Total execution time: 00:00:00.056

Now we'll create POLYGON spatial types using the CellBoundariesWithGeometryStrings data.

In [17]:
Use MouseHypothalamus;

DROP TABLE IF EXISTS CellBoundariesWithPolygonsHead;
SELECT 
        id, 
        layer,
        feature_uid,
        feature_id,
        geometry::STGeomFromText(REPLACE(geometry_string, '"', ''),0) AS polygon
    INTO CellBoundariesWithPolygonsHead
    FROM CellBoundariesHeadWithGeometryStrings;

(6930 rows affected)

Total execution time: 00:00:11.874

In [ ]:
ALTER TABLE CellBoundariesWithPolygons
ADD CONSTRAINT cellBoundariesWithPolygons_id_PK PRIMARY KEY(id);

In [ ]:
ALTER TABLE CellBoundariesWithPolygonsHead
ADD CONSTRAINT cellBoundariesWithPolygonsHead_id_PK PRIMARY KEY(id);